## Non-IID-Dataset-Generator-Direc-Delta

THIS code supports researchers who study federated learning with non-iid datasets.

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
import torch

iid_data =  datasets.FashionMNIST(root='data/FMNIST',download=True,train=True) 
num_classes = len(iid_data.classes)
alpha = 1 # if alpha is small --> increasing non-iidness, or if alpha is large --> increasing iidness
num_worker = 10 # Number of client in federated learning


In [24]:
class Non_iid(Dataset): 
    def __init__(self, x, y):
        self.x_data = x.unsqueeze(1).to(torch.float32)
        self.y_data = y.to(torch.int64)
        self.batch_size = 32 # set batchsize in here
        self.cuda_available = torch.cuda.is_available()
        
    # Return the number of data 
    def __len__(self): 
        return len(self.x_data)
    
    # Sampling
    def __getitem__(self): 
        idx = np.random.randint(low = 0, high= len(self.x_data), size=self.batch_size) # random_index
        x = self.x_data[idx]
        y = self.y_data[idx]
        if self.cuda_available :
            return x.cuda(), y.cuda()
        else:
            return x, y
        

## Generating-Non-iid-Dataset

In [25]:

idx   = [ torch.where(iid_data.train_labels == i) for i in range(num_classes) ] 
data  = [ iid_data.data[idx[i][0]] for i in range(num_classes) ] 
label = [ torch.ones(len(data[i]))* i for i in range(num_classes)]

s = np.random.dirichlet(np.ones(num_classes)*alpha, num_worker)
data_dist = np.zeros((num_worker,num_classes))

for j in range(num_worker):
    data_dist[j] = ((s[j]*len(data[0])).astype('int') / (s[j]*len(data[0])).astype('int').sum() * len(data[0])).astype('int')
    data_num     = data_dist[j].sum()
    data_dist[j][np.random.randint(low=0,high=num_classes)] += ((len(data[0]) - data_num) )
    data_dist    = data_dist.astype('int')
    
X = []
Y = []
for j in range(num_worker):
    x_data = []
    y_data = []
    for i in range(num_classes):
        if data_dist[j][i] != 0:
            d_index = np.random.randint(low=0, high=len(data[i]), size=data_dist[j][i])
            x_data.append(data[i][d_index])
            y_data.append(label[i][d_index])
    X.append(torch.cat(x_data))
    Y.append(torch.cat(y_data))
Non_iid_dataset  = [Non_iid(X[j],Y[j]) for j in range(num_worker)]

In [27]:
# Sampling first client's train data
data_, label_ = Non_iid_dataset[0].__getitem__()